# Find proteins outside the 7q gain event (trans) that are commonly different between samples with and without the event

## Setup

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
CHROMOSOME = "7"
ARM = "q"
TRANS_OR_CIS = "trans"

ttest_results_file = f"{CHROMOSOME}{ARM}_{TRANS_OR_CIS}effects_ttest.tsv"

ttest_results = pd.\
read_csv(ttest_results_file, sep="\t").\
rename(columns={"Name": "protein"}).\
set_index("protein")

In [3]:
ttest_results

,luad_Database_ID,lscc_Database_ID,brca_Database_ID,ccrcc_Database_ID,ovarian_Database_ID,brca_pvalue,ccrcc_pvalue,colon_pvalue,endometrial_pvalue,gbm_pvalue,...,luad_pvalue,ovarian_pvalue,brca_diff,ccrcc_diff,colon_diff,endometrial_diff,gbm_diff,lscc_diff,luad_diff,ovarian_diff
protein,,,,,,,,,,,,,,,,,,,,,
A1BG,NP_570602.2,NP_570602.2,NP_570602.2,NP_570602.2,NP_570602,0.999778,0.988830,0.974877,0.964232,0.855141,...,0.891168,0.754875,0.186072,0.046274,0.039473,-0.052113,-0.240198,0.194933,0.210642,0.363620
A2M,NP_000005.2|NP_001334353.1|NP_001334354.1|K4JD...,NP_000005.2|NP_001334353.1|NP_001334354.1,NP_000005.2,NP_000005.2,NP_000005,0.999778,0.941075,0.801544,0.923133,0.708471,...,0.915763,0.373553,0.171110,-0.216843,-0.163616,-0.123513,-0.464235,-0.323056,-0.196892,0.832943
A2ML1,NaN,NP_653271.2|NP_001269353.1,NP_653271.2|NP_001269353.1,NaN,NaN,0.999778,NaN,NaN,0.777623,NaN,...,NaN,NaN,-0.650888,NaN,NaN,0.765244,NaN,0.767315,NaN,NaN
AAAS,NP_056480.1|NP_001166937.1,NP_056480.1|NP_001166937.1,NP_056480.1|NP_001166937.1,NP_056480.1,NP_056480,0.999778,0.999335,0.944919,0.320554,0.999687,...,0.891168,0.947995,0.061091,-0.002386,-0.038812,-0.253439,0.000459,-0.083002,0.124254,-0.031490
AACS,NP_076417.2|NP_001306769.1|NP_001306768.1,NP_076417.2|NP_001306769.1|NP_001306768.1,NP_076417.2|NP_001306769.1|NP_001306768.1,NP_076417.2,NP_076417,0.999778,0.947935,0.876832,0.708236,0.855141,...,0.951468,0.915959,-0.353840,-0.117322,0.070477,0.282380,0.112100,-0.217657,0.166744,-0.128495
AAGAB,NP_078942.3|NP_001258814.1,NP_078942.3|NP_001258814.1,NP_078942.3|NP_001258815.1,NP_078942.3,NP_078942,0.999778,0.999335,0.985110,0.968236,0.862070,...,0.813472,0.844525,-0.198937,-0.006687,-0.008553,0.025021,0.080916,0.210252,0.253923,0.141799
AAK1,NaN,NP_055726.3,NP_055726.3,NP_055726.3,NP_055726,0.999778,0.999335,0.537005,0.868958,0.848386,...,NaN,0.729334,-0.207773,0.006018,-0.134425,-0.078399,0.174426,-0.075826,NaN,0.216391
AAMDC,NP_001303889.1|NP_001350493.1|NP_001303886.1|N...,NP_001303889.1|NP_001350493.1,NP_078960.1|NP_001303886.1|NP_001303887.1,NaN,NP_078960,0.999778,NaN,0.830773,0.889828,0.859841,...,0.910611,0.910957,-0.300751,NaN,0.109256,0.138844,-0.162048,-0.147423,0.286190,-0.153092
AAMP,NP_001289474.1|NP_001078.2,NP_001289474.1|NP_001078.2,NP_001289474.1|NP_001078.2,NP_001078.2,NP_001078,0.999778,0.941075,0.806242,0.565183,0.876759,...,0.732732,0.891168,-0.327191,-0.121910,-0.088067,-0.428287,-0.078436,-0.568333,-0.306869,-0.113868


## Reshape the input dataframe
We want to get our table to have these columns:
- cancer_type
- protein
- Database_ID
- change
- p_value

Since some cancer types have database IDs and some don't, we'll slice out and reshape the info for each cancer type individually.

In [4]:
cancer_types = sorted(ttest_results.columns.to_series().str.split("_", n=1, expand=True)[0].unique())

long_results = pd.DataFrame()

for cancer_type in cancer_types:
    cancer_df = ttest_results.\
    loc[:, ttest_results.columns.str.startswith(cancer_type)].\
    dropna(axis="index", how="all").\
    reset_index(drop=False)
    
    # If the cancer type has database IDs, make a separate column that has them.
    # If not, create a column of NaNs (so that the tables all match)
    if f"{cancer_type}_Database_ID" in cancer_df.columns:
        cancer_df = cancer_df.rename(columns={f"{cancer_type}_Database_ID": "Database_ID"})
    else:
        cancer_df = cancer_df.assign(Database_ID=np.nan)
        
    # Rename the pvalue and diff columns to not have the cancer type
    cancer_df = cancer_df.rename(columns={
        f"{cancer_type}_pvalue": "adj_p",
        f"{cancer_type}_diff": "change"
    }).\
    assign(cancer_type=cancer_type)
    
    # Reorder the columns
    cancer_df = cancer_df[["cancer_type", "protein", "Database_ID", "adj_p", "change"]]
    
    # Append to the overall dataframe
    long_results = long_results.append(cancer_df)

# Drop duplicate rows and reset the index
long_results = long_results[~long_results.duplicated(keep=False)].\
reset_index(drop=True)

In [5]:
long_results

,cancer_type,protein,Database_ID,adj_p,change
0,brca,A1BG,NP_570602.2,0.999778,0.186072
1,brca,A2M,NP_000005.2,0.999778,0.171110
2,brca,A2ML1,NP_653271.2|NP_001269353.1,0.999778,-0.650888
3,brca,AAAS,NP_056480.1|NP_001166937.1,0.999778,0.061091
4,brca,AACS,NP_076417.2|NP_001306769.1|NP_001306768.1,0.999778,-0.353840
5,brca,AAGAB,NP_078942.3|NP_001258815.1,0.999778,-0.198937
6,brca,AAK1,NP_055726.3,0.999778,-0.207773
7,brca,AAMDC,NP_078960.1|NP_001303886.1|NP_001303887.1,0.999778,-0.300751
8,brca,AAMP,NP_001289474.1|NP_001078.2,0.999778,-0.327191
9,brca,AAR2,NP_001258803.1,0.999778,0.166339


## Select the proteins with a significant change

In [6]:
prots = long_results[long_results["adj_p"] <= 0.05].reset_index(drop=True)

In [7]:
prots.groupby("cancer_type").count()[["protein"]]

,protein
cancer_type,
endometrial,1
lscc,1
ovarian,2


## Find how many cancers each protein was different in

In [8]:
def make_simple_change(change_val):
    if change_val == 0:
        return 0
    if change_val > 0:
        return 1
    if change_val < 0:
        return -1

prots = prots.assign(
    simplified_change=prots["change"].apply(make_simple_change)
)

In [9]:
prots

,cancer_type,protein,Database_ID,adj_p,change,simplified_change
0,endometrial,WDR33,NaN,0.033078,-0.375755,-1
1,lscc,TOE1,NP_079353.3,0.007812,-0.850758,-1
2,ovarian,PRKCSH,NP_001276031,0.001557,0.959957,1
3,ovarian,PRKCSH,NP_001276032,0.001557,0.955917,1


In [10]:
prots_summary = prots.groupby("protein").agg(**{
    "cancers": ("cancer_type", lambda x: x.sort_values().drop_duplicates(keep="first").tolist()),
    "mean_simp_change": ("simplified_change", np.mean)
})

prots_summary = prots_summary.\
assign(
    num_cancers=prots_summary["cancers"].apply(len),
    tmp_sort=prots_summary["cancers"].apply(lambda x: "".join(x))
).\
sort_values(by=["num_cancers", "tmp_sort"], ascending=[False, True]).\
drop(columns="tmp_sort")

prots_summary

,cancers,mean_simp_change,num_cancers
protein,,,
WDR33,[endometrial],-1,1
TOE1,[lscc],-1,1
PRKCSH,[ovarian],1,1


In [11]:
prots_summary[prots_summary["cancers"].apply(lambda x: "gbm" in x)]

,cancers,mean_simp_change,num_cancers
protein,,,


## Save results

In [12]:
prots_summary = prots_summary.assign(
    cancers=prots_summary["cancers"].apply(lambda x: "_".join(x))
)

output_file = f"pancancer_summary_{CHROMOSOME}{ARM}_{TRANS_OR_CIS}.tsv"
prots_summary.to_csv(output_file, sep="\t")